
---

## 📂 **5. Docker Volumes & Bind Mounts**

---

### 📦 **What are Docker Volumes?**

Volumes are **persistent storage** for Docker containers. They store data **outside the container’s writable layer**, so data **is not lost when the container is removed**.

🔄 Use volumes to:

* Persist database files, logs, uploads
* Share data between containers
* Avoid bloated container images

---

### 🧱 **Types of Volumes**

| Type            | Description                                                   | Use Case                               |
| --------------- | ------------------------------------------------------------- | -------------------------------------- |
| **Anonymous**   | Auto-created by Docker with random name                       | Quick tests or temp data               |
| **Named**       | Custom-named and managed via Docker CLI                       | Shared data across multiple containers |
| **Bind Mounts** | Link **host machine directories** directly into the container | Mount local source code or configs     |

---

### 🛠️ **Commands Cheat Sheet**

| Command                       | What it Does                  |
| ----------------------------- | ----------------------------- |
| `docker volume create myvol`  | Create named volume `myvol`   |
| `docker volume inspect myvol` | View details about the volume |
| `docker volume ls`            | List all volumes              |
| `docker volume rm myvol`      | Remove a specific volume      |
| `docker volume prune`         | Delete all unused volumes     |

---

### 🔗 **Using Volumes & Mounts in `docker run`**

| Feature       | Syntax Example                       | Notes                              |
| ------------- | ------------------------------------ | ---------------------------------- |
| Anonymous Vol | `docker run -v /data busybox`        | No name — Docker manages it        |
| Named Volume  | `docker run -v myvol:/data busybox`  | Re-usable across containers        |
| Bind Mount    | `docker run -v $(pwd):/app -it node` | Mount current host dir into `/app` |

> 💡 Always prefer **named volumes** for production, **bind mounts** for development.

---

